In [3]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import yfinance as yf

In [4]:
data = pd.DataFrame()
data = data.ta.ticker('EURUSD=x', period='5y')

In [5]:
df = data.copy()
df.ta.macd(append=True)
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9'],
      dtype='object')

In [6]:
df['signal'] = df.iloc[:, -3] > df.iloc[:, -1] # iloc[:, -n] select columns n from last
signal_vectorbt = df.ta.tsignals(df.signal, asbool=True, append=True)

In [7]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,signal,TS_Trends,TS_Trades,TS_Entries,TS_Exits
Date,,,,,,,,,,,,,,,
2016-09-26,1.123974,1.127904,1.122200,1.123924,0,0,0,NaN,NaN,NaN,False,False,0,False,False
2016-09-27,1.125075,1.126001,1.119400,1.124986,0,0,0,NaN,NaN,NaN,False,False,0,False,False
2016-09-28,1.121831,1.123722,1.118401,1.121957,0,0,0,NaN,NaN,NaN,False,False,0,False,False
2016-09-29,1.122460,1.125000,1.119946,1.122498,0,0,0,NaN,NaN,NaN,False,False,0,False,False
2016-09-30,1.122208,1.125000,1.115400,1.121957,0,0,0,NaN,NaN,NaN,False,False,0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-20,1.172883,1.173709,1.170056,1.172993,0,0,0,-0.000401,-0.000774,0.000373,False,False,0,False,False
2021-09-21,1.172567,1.174895,1.171591,1.173020,0,0,0,-0.000912,-0.001028,0.000116,False,False,0,False,False
2021-09-22,1.172608,1.174453,1.171715,1.172292,0,0,0,-0.001360,-0.001181,-0.000179,False,False,0,False,False


In [8]:
df['action_price'] = df['Open'].shift(-1)

In [9]:
df.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,signal,TS_Trends,TS_Trades,TS_Entries,TS_Exits,action_price
Date,,,,,,,,,,,,,,,,
2021-09-20,1.172883,1.173709,1.170056,1.172993,0,0,0,-0.000401,-0.000774,0.000373,False,False,0,False,False,1.172567
2021-09-21,1.172567,1.174895,1.171591,1.173020,0,0,0,-0.000912,-0.001028,0.000116,False,False,0,False,False,1.172608
2021-09-22,1.172608,1.174453,1.171715,1.172292,0,0,0,-0.001360,-0.001181,-0.000179,False,False,0,False,False,1.168893
2021-09-23,1.168893,1.175101,1.168419,1.168770,0,0,0,-0.001976,-0.001437,-0.000538,False,False,0,False,False,1.174178
2021-09-24,1.174178,1.174812,1.170166,1.174026,0,0,0,-0.002017,-0.001183,-0.000834,False,False,0,False,False,NaN


In [11]:
trades_table = df.iloc[:, -5:][df.TS_Trades != 0]
trades_table

,TS_Trends,TS_Trades,TS_Entries,TS_Exits,action_price
Date,,,,,
2016-11-10,True,1,True,False,1.088969
2016-11-15,False,-1,False,True,1.072616
2016-12-01,True,1,True,False,1.066325
2016-12-16,False,-1,False,True,1.044823
2016-12-27,True,1,True,False,1.046244
...,...,...,...,...,...
2021-08-10,False,-1,False,True,1.171976
2021-08-17,True,1,True,False,1.170960
2021-08-18,False,-1,False,True,1.170385


In [12]:
trades_table['return'] = trades_table['action_price'].pct_change()
trades_table

,TS_Trends,TS_Trades,TS_Entries,TS_Exits,action_price,return
Date,,,,,,
2016-11-10,True,1,True,False,1.088969,NaN
2016-11-15,False,-1,False,True,1.072616,-0.015017
2016-12-01,True,1,True,False,1.066325,-0.005865
2016-12-16,False,-1,False,True,1.044823,-0.020165
2016-12-27,True,1,True,False,1.046244,0.001360
...,...,...,...,...,...,...
2021-08-10,False,-1,False,True,1.171976,-0.007899
2021-08-17,True,1,True,False,1.170960,-0.000867
2021-08-18,False,-1,False,True,1.170385,-0.000492


In [13]:
trades_table['return'] = trades_table.loc[trades_table.TS_Exits==True]
trades_table

,TS_Trends,TS_Trades,TS_Entries,TS_Exits,action_price,return
Date,,,,,,
2016-11-10,True,1,True,False,1.088969,NaN
2016-11-15,False,-1,False,True,1.072616,False
2016-12-01,True,1,True,False,1.066325,NaN
2016-12-16,False,-1,False,True,1.044823,False
2016-12-27,True,1,True,False,1.046244,NaN
...,...,...,...,...,...,...
2021-08-10,False,-1,False,True,1.171976,False
2021-08-17,True,1,True,False,1.170960,NaN
2021-08-18,False,-1,False,True,1.170385,False


In [16]:
compare = []
trade_log = []
for i in range(11, 15):
    for j in range(21, 25):
        df = data.copy() # reset dataframe
        df.ta.macd(fast=i, slow=j, append=True) # run macd
        df['signal'] = df.iloc[:, -3] > df.iloc[:, -1] # create signal
        signal_vectorbt = df.ta.tsignals(df.signal, asbool=True, append=True) # create TS table
        
        df['action_price'] = df['Open'].shift(-1) # action on next open
        
        # create trade log
        trades_table = df.iloc[:, -5:][df.TS_Trades != 0]
        trades_table['return'] = trades_table['action_price'].pct_change()
        trades_summary = trades_table.loc[trades_table.TS_Exits==True]
        
        # select data for compare
        a = trades_summary['return']
        trade_log.append(a)
        
        b = [a.mean(), a.median(), a.min(), (i, j)]
        compare.append(b)
        

In [18]:
result = pd.DataFrame(compare, columns=['Avg Return', 'Median', 'Worst Trade Return', 'MACD Parameter Set']).set_index('MACD Parameter Set', drop=True)
result

,Avg Return,Median,Worst Trade Return
MACD Parameter Set,,,
"(11, 21)",-0.001324,-0.003652,-0.032026
"(11, 22)",-0.001581,-0.003843,-0.033358
"(11, 23)",-0.001685,-0.003843,-0.029347
"(11, 24)",-0.001469,-0.003652,-0.026708
"(12, 21)",-0.001728,-0.003843,-0.033895
"(12, 22)",-0.001583,-0.003652,-0.033358
"(12, 23)",-0.001590,-0.003079,-0.029347
"(12, 24)",-0.001517,-0.003366,-0.026708
"(13, 21)",-0.001714,-0.003079,-0.033895


In [19]:
result[result['Median'] == result['Median'].max()]

,Avg Return,Median,Worst Trade Return
MACD Parameter Set,,,
"(14, 23)",-0.001074,-0.002457,-0.029347


In [ ]:
# next step, calculate with fee and backtest from normal to crisis event.